In [1]:
%matplotlib inline
# -*- coding: utf-8 -*-
"""
Exemple de MNIST avec PyTorch
Exemple avec ResNet5 (5 couches convolutives) pour 28x28
Exploitation des 60000 données d'entrainement et des données de test
Augmentation des données d'entrainement
Version avec GPU
"""
import torch
torch.manual_seed(0) # Pour résultats reproductibles

# Fonction J d'entropie croisée
import torch.nn.functional as F
fonction_cout = F.cross_entropy

def taux_bonnes_predictions(lot_Y_predictions, lot_Y):
    predictions_categorie = torch.argmax(lot_Y_predictions, dim=1)
    return (predictions_categorie == lot_Y).float().mean()

from torch import nn
# Définition de l'architecture du RNA

class BlocResiduel2d(nn.Module):
    """ Représente un bloc de base de l'architecture Resnet
    nb_canaux_in : nombre de canaux de l'entrée X
    nb_canaux_out : nombre de canaux de la sortie Y
    pas : le pas de la première convolution conv1
    """
    def __init__(self, nb_canaux_in, nb_canaux_out,pas=1):
        super().__init__()
        self.conv1 = nn.Conv2d(nb_canaux_in, nb_canaux_out,kernel_size=3, padding=1, stride=pas)
        self.conv2 = nn.Conv2d(nb_canaux_out, nb_canaux_out,kernel_size=3, padding=1)
        if nb_canaux_in != nb_canaux_out:
            self.conv3 = nn.Conv2d(nb_canaux_in, nb_canaux_out,kernel_size=1, stride=pas)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(nb_canaux_out)
        self.bn2 = nn.BatchNorm2d(nb_canaux_out)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, lot_X):
        lot_Y_predictions = F.relu(self.bn1(self.conv1(lot_X)))
        lot_Y_predictions = self.bn2(self.conv2(lot_Y_predictions))
        
        # si le nombre de canaux de la sortie est différent de l'entrée
        # la convolution 1x1 fait la conversion de l'entrée X
        if self.conv3:
            lot_X = self.conv3(lot_X)

        lot_Y_predictions += lot_X
        return F.relu(lot_Y_predictions)
    
# Resnet5 simplifié sans la première convolution pour résolution MNIST 28x28
modele = nn.Sequential(BlocResiduel2d(1,32), BlocResiduel2d(32,64) ,# ->(N,64,28,28)
                   nn.AdaptiveMaxPool2d((1,1)), nn.Flatten(), nn.Linear(64,10))

from torch import optim
optimiseur = optim.SGD(modele.parameters(), lr=0.01)

import torchvision
from torchvision import datasets, models, transforms

transformations_ent = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Normalisation mais sans augmentation pour la validation
transformations_valid = transforms.Compose([
    transforms.ToTensor()
])

#Chargement des données
ds_ent = torchvision.datasets.MNIST(root = "./data", train = True, download = True, transform = transformations_ent)
ds_valid = torchvision.datasets.MNIST(root = "./data", train = False, download = True, transform = transformations_valid)

#Création du DataLoader avec le dataset
dl_ent = torch.utils.data.DataLoader(ds_ent, batch_size=100, shuffle = True)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=100)

# Déterminer si un GPU est disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Entrainement sur ',device)

# Placer le modèle en mode GPU si possible
modele = modele.to(device)

import time
def entrainer_GPU(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10):

    debut = time.time()

    # Listes pour les métriques par epoch
    liste_cout_moyen_ent = []
    liste_taux_moyen_ent = []
    liste_cout_moyen_valid = []
    liste_taux_moyen_valid = []
    
    # Boucle d'apprentissage
    for epoch in range(nb_epochs):
        cout_total_ent = 0 # pour cumuler les couts par mini-lot
        taux_bonnes_predictions_ent = 0 # pour cumuler les taux par mini-lot
        modele.train() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        
        # Boucle d'apprentissage par mini-lot pour une epoch
        for lot_X, lot_Y in dl_ent:
            
            # Les données doivent être placés en GPU pour être compatibles avec le modèle
            lot_X = lot_X.to(device)
            lot_Y = lot_Y.to(device)

            optimiseur.zero_grad() # Remettre les dérivées à zéro
            lot_Y_predictions = modele(lot_X) # Appel de la méthode forward
            cout = fonction_cout(lot_Y_predictions, lot_Y)
            cout.backward() # Calcul des gradiants par rétropropagation
            with torch.no_grad():
                cout_total_ent +=cout
                taux_bonnes_predictions_ent += taux_bonnes_predictions(lot_Y_predictions, lot_Y)
            optimiseur.step() # Mise à jour des paramètres
            # scheduler.step()
        # Calculer les moyennes par mini-lot
        with torch.no_grad():
            cout_moyen_ent = cout_total_ent/len(dl_ent)
            taux_moyen_ent = taux_bonnes_predictions_ent/len(dl_ent)
       
        modele.eval() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        with torch.no_grad():
            # Les données de validation doivent aussi être placés en GPU
            cout_valid = sum(fonction_cout(modele(lot_valid_X.to(device)), lot_valid_Y.to(device)) for lot_valid_X, lot_valid_Y in dl_valid)
            taux_bons_valid = sum(taux_bonnes_predictions(modele(lot_valid_X.to(device)), lot_valid_Y.to(device)) for lot_valid_X, lot_valid_Y in dl_valid)
        cout_moyen_valid = cout_valid/len(dl_valid)
        taux_moyen_valid = taux_bons_valid/len(dl_valid)
        print(f'-------- > epoch {epoch+1}:  coût moyen entraînement = {cout_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  taux moyen entraînement = {taux_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  coût moyen validation = {cout_moyen_valid}')
        print(f'-------- > epoch {epoch+1}:  taux moyen validation = {taux_moyen_valid}')
    
        liste_cout_moyen_ent.append(cout_moyen_ent)
        liste_taux_moyen_ent.append(taux_moyen_ent)
        liste_cout_moyen_valid.append(cout_moyen_valid)
        liste_taux_moyen_valid.append(taux_moyen_valid)
        
        temps_ecoule = time.time() - debut
        print('Temps écoulé : {:.0f}m {:.0f}s'.format(temps_ecoule // 60, temps_ecoule % 60))

    
    # Affichage du graphique d'évolution des métriques par epoch
    import numpy as np
    import matplotlib.pyplot as plt
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_ent,label='Erreur entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_valid,label='Erreur validation')
    plt.title("Evolution du coût")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='upper center')
    plt.show()
        
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_ent,label='Taux bonnes réponses entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_valid,label='Taux bonnes réponses validation')
    plt.title("Evolution du taux")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='center')
    plt.show()

entrainer_GPU(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10)

Entrainement sur  cpu


KeyboardInterrupt: 